In [19]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep 
# from tqdm import tqdm # чтобы показать бар заполняемости прогресса

from datetime import datetime, timedelta
from openpyxl import load_workbook # для работы с excel

In [6]:
# Параметры поездки

# Откуда
origin = 'москва'
# Куда
destination = 'псков'

# Дата отправления
# Введите дату отправления в формате "dd.mm.yyyy"
departure_date = '16.03.2023'

# Дата возвращения
# Введите дату возвращения в формате "dd.mm.yyyy"
return_date = '17.03.2023'

# Введите желаемое время прилета в город командировки формате 'hh:00'
arrival_start_time = '08:00'
arrival_end_time = '09:00'
# Введите желаемое время прилета обратно в Москву из города командировки формате 'hh:00'
returning_start_time = '19:00'
returning_end_time = '23:00'

# Выберите тип вагона из ['Сидячий', 'Плацкарт', 'Купе', 'СВ', 'Люкс']
w_type = 'Купе'

# Выберите рейтинг поезда: 'Высокий' или нет
rating = 'ВЫсокий'

In [20]:
# Save our trip data to Excel report file
wb = load_workbook('C:\Evhen\BSNS\ViK_Medical\Отчеты.xlsx')
ws = wb['Sheet1']

city_col = 3
date_col = 5
rows = [x for x in range (2, 150)]

for row in rows:
    # fill first empty row
    if (ws.cell(row=row, column=city_col).value==None) and (ws.cell(row=row, column=date_col).value==None):
        ws.cell(row=row, column=city_col).value = destination.capitalize()
        ws.cell(row=row, column=date_col).value = return_date
        break

wb.save('C:\Evhen\BSNS\ViK_Medical\Отчеты.xlsx')

In [2]:
# to exclude DeprecationWarning
service = Service('C:/Users/eadub/chromedriver_win32/chromedriver.exe')
# Запускаем Chrome driver
browser = Chrome(service=service)
browser.maximize_window()
browser.implicitly_wait(10)

In [4]:
url = 'https://www.tutu.ru/'
browser.get(url)
sleep(5)

In [5]:
# Changing to trains tab
trains = browser.find_element(By.CSS_SELECTOR, "div.t-ttl_third:nth-child(2) > div:nth-child(3)")
trains.click()

In [7]:
# Заполняем поле Откуда
input_origin = browser.find_element(By.CSS_SELECTOR, ".m-train > form:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > input:nth-child(1)")
input_origin.click()
input_origin.send_keys(origin)

# Зполняем поле Куда
input_destination = browser.find_element(By.CSS_SELECTOR, ".m-train > form:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > input:nth-child(1)")
input_destination.click()
input_destination.send_keys(destination)

In [8]:
# Заполняем Дату отправления
input_dep_date = browser.find_element(By.CSS_SELECTOR, ".m-train > form:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(4) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > input:nth-child(1)")
input_dep_date.click()
input_dep_date.click()
input_dep_date.send_keys(departure_date)

In [9]:
try:
    ret_date_button = browser.find_element(By.CSS_SELECTOR, ".date_end_trigger > span:nth-child(1)")
    ret_date_button.click()
except:
    pass

# Заполняем Дату возвращения
input_ret_date = browser.find_element(By.CSS_SELECTOR, ".j-travel_date_date_second > div:nth-child(1) > input:nth-child(1)")
input_ret_date.click()
input_ret_date.send_keys(return_date)

In [10]:
# Start searching
search_button = browser.find_element(By.CSS_SELECTOR, "div.button_wrp:nth-child(6) > button:nth-child(1) > span:nth-child(1) > span:nth-child(3)")
search_button.click()
sleep(10)

In [11]:
# Filter only available tickets
try:
    # clicking on the checkboxes for both departure and return    
    for i in range(1,3):
        browser.execute_script("arguments[0].click();", wait(browser,5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"div.filter__wrp__3HXHK:nth-child({i}) > div:nth-child(2) > div:nth-child(6) > div.filter__field__36deT[data-ti='isPossibleOnly']"))))
except:
    pass

In [12]:
# Filter type of wagon
try:
    # clicking on the needed wagon type for both departure and return
    for i in range(1, 3):
        # choosing type
        for j in range(2,7):
            type_checkbox = wait(browser,5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"div.filter__wrp__3HXHK:nth-child({i}) > div:nth-child(2) > div:nth-child(1) > div:nth-child({j}) > div:nth-child(1) > span:nth-child(2)")))
            if type_checkbox.text == w_type:
    #             type_checkbox.click()
                    browser.execute_script("arguments[0].click();", type_checkbox)
    sleep(2)
except:
    pass

In [13]:
# Filter train rating
try:
    # choosing rating for both departure and return
    for i in range(1, 3):
        if rating.lower() == 'высокий':
            rating_tutu = wait(browser,5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"div.filter__wrp__3HXHK:nth-child({i}) > div:nth-child(2) > div:nth-child(3) > div:nth-child(2) > div:nth-child(1) > span:nth-child(2) > span:nth-child(1)")))
            rating_tutu.click()
        else:
            rating_tutu = wait(browser,5).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f"div.filter__wrp__3HXHK:nth-child({i}) > div:nth-child(2) > div:nth-child(3) > div:nth-child(4) > div:nth-child(1) > span:nth-child(2) > span:nth-child(1)")))
            rating_tutu.click()
        sleep(2)
except:
    pass

In [14]:
def to_time_format(t): 
    t = datetime.strptime(t, "%H:%M").time()
    return t

In [15]:
# Choosing ticket to the destination city
to = browser.find_element(By.CSS_SELECTOR, ".l-content > div:nth-child(2) > div:nth-child(4) > div:nth-child(1) > div:nth-child(2)")
# counting all available tickets to the destination city
els = to.find_elements(By.CSS_SELECTOR, "span.schedule_time")
num_to = len(els)

# destination city
des_tutu = to.find_elements(By.CSS_SELECTOR, "span.route_block[data-ti='route_arrival_station']")
# arrival time (to the destination city)
arr_time_tutu = to.find_elements(By.CSS_SELECTOR, "span.schedule_time")
# choose ticket button 
choose_button = to.find_elements(By.CLASS_NAME, "b-button__standart")

for i in range(num_to):
    # exclude filtered tickets (their values got empty)
    if des_tutu[i].text != '':
        # choosing needed time
        if to_time_format(arrival_start_time)<=to_time_format(arr_time_tutu[i].text)<=to_time_format(arrival_end_time):
#             print(des_tutu[i].text)
#             print(arr_time_tutu[i].text)
#             print(choose_button[i].text)
            try:
                choose_button[i].click()
                break
            except:
                pass
        # Если в выбранный изначально временной диапазон билетов нет, сдвигаем время прибытия на час раньше 
        elif ((datetime.strptime(arrival_start_time, "%H:%M") - timedelta(hours=1)).time() 
              <= to_time_format(arr_time_tutu[i].text)
              <= to_time_format(arrival_end_time)):
            try:
                choose_button[i].click()
                break
            except:
                pass
sleep(2)

In [16]:
# Choosing ticket back to Moscow
back = browser.find_element(By.CSS_SELECTOR, ".l-content > div:nth-child(3) > div:nth-child(4) > div:nth-child(1) > div:nth-child(2)")
# counting all available tickets back to origin city
els = back.find_elements(By.CSS_SELECTOR, "div.departure_time")
num_back = len(els)

# origin city
origin_tutu = back.find_elements(By.CSS_SELECTOR, "span.route_block[data-ti='route_arrival_station']")
# returning time (back to Moscow)
ret_time_tutu = back.find_elements(By.CLASS_NAME, "departure_time")
# choose ticket button 
choose_button = back.find_elements(By.CLASS_NAME, "b-button__standart")

for i in range(num_back):
    # there're some empty values - i exclude them
    if origin_tutu[i].text != '':
        # choosing needed time
        if to_time_format(returning_start_time)<=to_time_format(ret_time_tutu[i].text)<=to_time_format(returning_end_time):
#             print(i)
#             print(origin_tutu[i].text)
#             print(ret_time_tutu[i].text)
#             print(choose_button[i].text)
            try:
                choose_button[i].click()
                break
            except:
                pass
        # Если в выбранный изначально временной диапазон билетов нет, сдвигаем интервал на час 
        elif ((datetime.strptime(returning_start_time, "%H:%M") - timedelta(hours=1)).time() 
              <= to_time_format(arr_time_tutu[i].text)
              <= (datetime.strptime(returning_end_time, "%H:%M") + timedelta(hours=1)).time()):
            try:
                choose_button[i].click()
                break
            except:
                pass
sleep(2)

In [17]:
# Choosing suitable back and forth tickets 
fin_choose = browser.find_element(By.CSS_SELECTOR, "._blue > span:nth-child(1) > span:nth-child(2)")
fin_choose.click()